In [15]:
import os
import pandas as pd
import numpy  as np
from scipy.fft    import dct, idct
from scipy.signal import butter, savgol_filter, ellip, filtfilt, detrend
from matplotlib   import pylab as plt

## Loading data

In [16]:
# paths were the raw data is
root_path = 'data/'
data_path = os.path.join(root_path, 'Experiment')
shimmer_path  = os.path.join(data_path, 'shimmer')

# paths were the preprocessed data should be
out_path = os.path.join(root_path, 'Processed')
emg_out_path = os.path.join(out_path, 'EMG')

# check if the output folders exist and create them if necessary
for folder in [out_path, emg_out_path]:
    if not os.path.exists(folder):
        os.mkdir(folder)

# list all files (ignoring the hidden macos file)
shimmer_files = [x for x in os.listdir(shimmer_path) if x.endswith('.csv')]

In [17]:
shimmer_files

['04_EDA_SS2023_Exp1_Session1_Shimmer_D210_Calibrated_SD.csv',
 '02_EMG_SS2023_Exp1_Session1_Shimmer_F16C_Calibrated_SD.csv',
 '04_EMG_SS2023_Exp1_Session1_Shimmer_EC93_Calibrated_SD.csv',
 'Trigger_SS2023_Exp1_Session1_Shimmer_895A_Calibrated_SD.csv',
 '03_EDA_SS2023_Exp1_Session1_Shimmer_86D4_Calibrated_SD.csv',
 '03_EMG_SS2023_Exp1_Session1_Shimmer_EC8E_Calibrated_SD.csv',
 '01_EMG_SS2023_Exp1_Session1_Shimmer_F0BA_Calibrated_SD.csv']

In [18]:
# set the specific path
emg_path = shimmer_files[2]
trigger_path = shimmer_files[3]

In [19]:
emg_path, trigger_path

('04_EMG_SS2023_Exp1_Session1_Shimmer_EC93_Calibrated_SD.csv',
 'Trigger_SS2023_Exp1_Session1_Shimmer_895A_Calibrated_SD.csv')

## Preprocessing

In [43]:
# Reshape a numpy array 'a' of shape (n, x) to form shape((n - window_size), window_size, x))
def rolling_window(a, window, step_size):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1 - step_size + 1, window)
    strides = a.strides + (a.strides[-1] * step_size,)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [63]:
# all 3 functions taken from matlab implementation after Anne Grübler
def notch_filter(data, notch, width, fs):
    fa = (notch - width) / (fs/2)
    fb = (notch + width) / (fs/2)
    Wn = np.array([fa, fb])
    order = 4
    b, a = butter(order, Wn, btype='stop')
    return filtfilt(b, a, data)

def elliptic_filter(data, flow, fhigh, fs):
    Wn    = np.array([flow, fhigh]) * (2/fs)
    order = 4
    maxRipple      = 0.1
    minAttenuation = 40
    b, a  = ellip(order, maxRipple, minAttenuation, Wn, btype='stop')
    return filtfilt(b, a, data)

def local_filter(data, notch, width, flow, fhigh, fs):
    signal = data
    if (len(notch) > 1):
        for kk in range(len(notch)):
            signal = notch_filter(signal, notch[kk], width[kk], fs)
    else:
        signal = notch_filter(signal, notch, width, fs)
    signal = elliptic_filter(signal, flow, fhigh, fs)
    return detrend(signal)

In [64]:
fs      = 1024 # emg sampling frequency
f_notch = np.array([50, 100, 150, 200, 250, 300]) # notch frequencies to filter out
width   = np.ones(len(f_notch)) * 3;              # width/2 of each notch
cutlow  = 5    # lower cut-off frequency for elliptic filter
cuthigh = 350  # upper cut-off frequency for elliptic filter


filepath = os.path.join(shimmer_path, emg_path)
df       = pd.read_csv(filepath, skiprows=[0, 2])

In [65]:
df.columns

Index(['Shimmer_EC93_TimestampSync_Unix_CAL',
       'Shimmer_EC93_ECG_EMG_Status1_CAL', 'Shimmer_EC93_EMG_CH1_24BIT_CAL',
       'Shimmer_EC93_EMG_CH2_24BIT_CAL', 'Unnamed: 4'],
      dtype='object')

In [66]:
df_ch1 = df['Shimmer_EC93_EMG_CH1_24BIT_CAL']
df_ch2 = df['Shimmer_EC93_EMG_CH2_24BIT_CAL']

In [67]:
# filter and save it into the data frame loaded
df_ch1 = local_filter(df_ch1, f_notch, width, cutlow, cuthigh, fs)
df_ch2 = local_filter(df_ch2, f_notch, width, cutlow, cuthigh, fs)

In [ ]:
# save data frame with pre-processed data to new file
df.to_csv(os.path.join(emg_out_path, emg_path), index=False)